In [1]:
import sys 
import torch
sys.path.insert(1, 'MolT5/baselines')

import numpy as np
import dataloader

from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [3]:
ALLOWED_ATOM_TYPES = ['C', 'O', 'N', 'F', 'S', 'Cl', 'Br', 'I', 'P']

In [10]:
geom = torch.load('./datasets/geom_train.pt')

/tmp/ipykernel_1211566/1359271894.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  geom = torch.load('./datasets/geom_train.pt')


In [6]:
geom_cut = geom[:10000]
torch.save(geom_cut, './datasets/geom_cut.pt')

In [4]:
geom_cut = torch.load('./datasets/geom_cut.pt')

/tmp/ipykernel_1211566/2042971286.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  geom_cut = torch.load('./datasets/geom_cut.pt')


In [ ]:
mol = geom_cut[0]
[ALLOWED_ATOM_TYPES[x] for x in mol['one_hot'].argmax(dim=-1)]

In [12]:
names = []
positions = []
atoms = []

for mol in tqdm(geom):
    names.append(mol['name'])
    positions.append(mol['positions'])
    atoms.append([ALLOWED_ATOM_TYPES[x] for x in mol['one_hot'].argmax(dim=-1)])

100%|██████████| 436471/436471 [01:57<00:00, 3708.03it/s]


In [15]:
strings_data = []

for mol in tqdm(geom):
    string = ''
    for pos, atom in zip(mol['positions'], [ALLOWED_ATOM_TYPES[x] for x in mol['one_hot'].argmax(dim=-1)]):
        x,y,z = pos
        string += f"{atom} {x:.3f} {y:.3f} {z:.3f} "
    strings_data.append(string)

100%|██████████| 436471/436471 [08:52<00:00, 819.21it/s]


In [1]:
for mol in geom:
    mol['bonds']
    break

NameError: name 'geom' is not defined

In [2]:
embs = torch.load('/home/user12/burov/unimol_embeddings_project/geom_train_embeddings.pt')

/tmp/ipykernel_1227971/2975242406.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embs = torch.load('/home/user12/burov/unimol_embeddings_project/geom_train_embeddings.p

In [ ]:
dummy_emb = torch.rand(10000, 10000, 64)

(2, 436471)

In [23]:
X_train, X_testval, y_train, y_testval = train_test_split(strings_data, names, random_state=42, train_size=0.9)
X_val, X_test, y_val, y_test = train_test_split(strings_data, names, random_state=42, train_size=0.5)

In [22]:
len(X_train)

327353

In [25]:
last = 0
with open('./MolT5/my_dataset/train.txt', 'w') as f:
    for i, (s, name) in tqdm(enumerate(zip(X_train, y_train))):
        f.write(f'CID{i:05}'+'\t'+name+' \t'+s+'\n')
        last = i
with open('./MolT5/my_dataset/test.txt', 'w') as f:
    for i, (s, name) in enumerate(zip(strings_data, names)):
        f.write(f'CID{i+last:05}'+'\t'+name+' \t'+s+'\n')
        last = i
with open('./MolT5/my_dataset/val.txt', 'w') as f:
    for i, (s, name) in enumerate(zip(strings_data, names)):
        f.write(f'CID{i+last:05}'+'\t'+name+' \t'+s+'\n')

    
with open('./dataforcid.txt', 'w') as f:
    for i, (s, name) in enumerate(zip(strings_data, names)):
        f.write(f'CID{i:05}'+'\t'+name+' \t'+s+'\n')

392823it [00:00, 414834.33it/s]


In [26]:
import pickle, csv

id2smi = {}
with open('./dataforcid.txt') as f:
    rdr = csv.DictReader(f, delimiter='\t', fieldnames=['cid','mol2vec','desc'])
    for row, smiles in zip(rdr, names):
        cid = row['cid']
        id2smi[cid] = smiles

with open('./MolT5/evaluation/cid_to_smiles.pkl', 'wb') as f:
    pickle.dump(id2smi, f)


In [1]:
from transformers import AutoTokenizer, T5Tokenizer

print("AutoTokenizer, slow:")
tok = T5Tokenizer.from_pretrained("google/t5-v1_1-base")
print(type(tok))

print("\nT5Tokenizer, slow:")
tok2 = T5Tokenizer.from_pretrained("google/t5-v1_1-base", use_fast=False)
print(type(tok2))

AutoTokenizer, slow:


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


<class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>

T5Tokenizer, slow:
<class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>


In [ ]:
import sys
sys.path.insert(1, './MolT5/baselines')
from main_transformer_caption2smiles import SmilesTokenizer 

In [20]:
geom[0]['one_hot'].argmax(-1), geom[0]['positions'].shape[0]

(tensor([0, 0, 0, 1, 2, 0, 2, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 5],
        device='cuda:0'),
 19)

In [23]:
from tqdm import tqdm
import torch, json

max_dist = 2.0
out = open("bond_prompts_small.jsonl", "w")

for mol in tqdm(geom[:10000]):                           # geom — это list(dict)
    xyz   = mol["positions"]        # (N, 3)
    types = mol["one_hot"].argmax(-1)           # (N,)

    # 1. полная матрица расстояний
    dmat = torch.cdist(xyz, xyz, p=2)           # (N, N)

    # 2. выбираем пары ≤ 2 Å (и не i==j)
    idx   = (dmat > 0) & (dmat <= max_dist)
    pairs = idx.nonzero(as_tuple=False)         # (M, 2)

    if len(pairs) == 0:
        continue

    # ────────────────── постройте dict → (i,j) → one-hot в bond_orders ──────────
    edge2order = {}
    ei, bo = mol["edge_index"], mol["bond_orders"]          # (E,2), (E,10)
    for k in range(ei.shape[0]):
        u, v = map(int, ei[k])
        edge2order[(u, v)] = bo[k]
    # undirected: сразу добавим обратное направление
        edge2order[(v, u)] = bo[k]

    # ────────────────── определяем label для каждой пары ≤ 2 Å ───────────────────
    labels = []
    for i, j in pairs:                                      # pairs — Tensor (M,2)
        vec = edge2order.get((int(i), int(j)))              # None, если нет связи
        if vec is None:
            bond = 0                                        # «no-bond»
        else:
            idx = vec.nonzero(as_tuple=True)[0].item()      # 0…9
            bond = {0:0, 1:1, 2:2, 3:3, 4:1}.get(idx, 0)    # сводим к 0-3
        labels.append(bond)


    # 4. строим prompt
    atom_map = ", ".join(f"{i}:{'CONHXBOSPFI'[types[i]]}"   # грубое сокращение; лучше RDKit
                         for i in range(len(types)))
    dist_map = "\n".join(f"[{int(i)},{int(j)}]={dmat[i,j]:.3f}"
                         for i, j in pairs)
    system = ("You are a chemist. Task: classify bond type (0 no-bond, 1 single, "
              "2 double, 3 triple). Return ONLY JSON list "
              "`[{\"pair\":[i,j],\"label\":n}, ...]` in the given order.")
    user   = f"Atoms (index→element): {atom_map}\nDistances (Å):\n{dist_map}"
    assistant = json.dumps(
        [{"pair":[int(i), int(j)], "label": int(l)}
         for (i, j), l in zip(pairs, labels)],
        separators=(",", ":"))

    json.dump({"messages":[
        {"role":"system", "content":system},
        {"role":"user",    "content":user},
        {"role":"assistant","content":assistant}
    ]}, out); out.write("\n")


100%|██████████| 10000/10000 [03:43<00:00, 44.76it/s]
